In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
demographics_event= pd.read_csv('/data/public/MLA/VCHAMPS-Quality/demographics_event_qual.csv').iloc[:,1:]

In [3]:
demographics_event['Event date'] = demographics_event['Event date'].apply(preprocess_data.preprocess_time_string)

In [4]:
marital_status = {'Married': 0, 'Divorced': 1, 'Never married': 1, 'Separated': 1, 'Single': 1, 'Unknown': float('nan'), 'Widowed': 1, 'Not specified (no value)': float('nan')}
ruca_category = {'Urban': 0,  'Rural': 1, 'Highly rural': 2, 'Not specified': float('nan')}

In [5]:
demographics_event['Marital status encoded'] = preprocess_data.preprocess_substitute_categories(marital_status, 
                                                                               demographics_event['Marital status'])
demographics_event['Ruca category encoded'] = preprocess_data.preprocess_substitute_categories(ruca_category, 
                                                                               demographics_event['Ruca category'])

In [6]:
outpatients_location = pd.read_csv('/home/vivi/FDA_datasets/outpatient_state_quality_admission.csv').iloc[:,1:]
demographics_event_location = demographics_event.merge(outpatients_location, how = 'left', on = 'Internalpatientid')

In [7]:
demographics_event_location = demographics_event_location.dropna(subset=['State']) # 

In [8]:
state_ruca = demographics_event_location.groupby('State')['Ruca category encoded'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [9]:
for index, row in demographics_event_location.iterrows():
    if pd.isna(row['Ruca category encoded']):
        demographics_event_location.at[index, 'Ruca category encoded'] = state_ruca[state_ruca['State'] == row['State']]['Ruca category encoded']

In [10]:
demographics_event_location

,Internalpatientid,Age at update,Event date,Marital status,Ruca category,Marital status encoded,Ruca category encoded,State
2,103133,100.560119,2017-08-28 17:22:05,Married,Urban,0.0,0.0,Rhode Island
4,105402,85.572665,2021-08-09 19:25:16,Separated,Urban,1.0,0.0,Florida
5,106818,52.030201,2018-01-15 00:55:22,Unknown,Urban,NaN,0.0,South Carolina
9,100012,74.603234,2024-02-12 11:34:10,Widowed,Urban,1.0,0.0,New Mexico
10,102788,73.531928,2024-06-13 22:04:26,Never married,Urban,1.0,0.0,Texas
...,...,...,...,...,...,...,...,...
992,84990,68.175906,2019-04-25 07:02:40,Married,Urban,0.0,0.0,Connecticut
993,85966,41.372840,2023-02-15 18:33:03,Never married,Rural,1.0,1.0,Texas
994,88042,68.306925,2014-11-16 04:20:57,Never married,Urban,1.0,0.0,Connecticut
997,93917,68.382026,2024-09-24 15:08:02,Married,Rural,0.0,1.0,Tennessee


In [11]:
demographics_event_location.to_csv('/home/vivi/FDA_datasets/demographics_event_quality_admission.csv')